In [1]:
%pip install fairlearn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import re

# Data manipulation 
import pandas as pd 
import numpy as np 

# Visualization 
import matplotlib.pyplot as plt
import seaborn as sns 

# Fairness 
from fairlearn.metrics import demographic_parity_difference  # type: ignore
# MongoDB 
from pymongo import MongoClient

# Data Quality Analysis: NovaCred Credit Applications
**Task Force:** Team DEGO
**Objective:** Evaluate, quantify, and remediate data quality issues in the `raw_credit_applications.json` dataset across 4 dimensions: Completeness, Consistency, Validity, and Accuracy.

## Phase 1: Data Ingestion & Flattening
The original dataset is provided in a nested JSON format (e.g., `applicant_info`, `financials`). To perform an effective Exploratory Data Analysis (EDA) using Pandas, our first step is to "flatten" this structure, transforming the nested keys into standard tabular columns. We will use the `pd.json_normalize()` function.

To facilitate the Data Scientist's "Proxy Discrimination Analysis", we must transform the nested `spending_behavior` array into standard numerical features. We will pivot the categories into individual columns (e.g., `spending_Rent`, `spending_Healthcare`), allowing for direct correlation and statistical testing against the `loan_approved` outcome.

In [3]:
print("=== PHASE 1: DATA INGESTION & ADVANCED FLATTENING ===")
file_path = '../data/raw/raw_credit_applications.json'

with open(file_path, 'r') as file:
    raw_data = json.load(file)

# 1. Standard Flattening
df = pd.json_normalize(raw_data)

# 2. Advanced Flattening: Explode spending_behavior immediately
if 'spending_behavior' in df.columns:
    print("Unpacking nested spending behaviors...")
    exploded = df.explode('spending_behavior')
    
    # Extract category and amount
    exploded['spend_category'] = exploded['spending_behavior'].apply(lambda x: x.get('category') if isinstance(x, dict) else None)
    exploded['spend_amount'] = exploded['spending_behavior'].apply(lambda x: x.get('amount') if isinstance(x, dict) else 0)
    
    # Pivot into separate columns
    spending_pivot = exploded.pivot_table(
        index='_id', 
        columns='spend_category', 
        values='spend_amount', 
        aggfunc='sum',
        fill_value=0
    )
    spending_pivot.columns = [f'spending_{col}' for col in spending_pivot.columns]
    
    # Merge back and drop the original nested column
    df = df.merge(spending_pivot, on='_id', how='left')
    df.drop(columns=['spending_behavior'], inplace=True)
    
    # Ensure any missing spending values are set to 0
    for col in spending_pivot.columns:
        df[col] = df[col].fillna(0)

print("Data successfully loaded and flattened.")
print(f"Dataset Shape: {df.shape[0]} records and {df.shape[1]} attributes.")
display(df.head(5))

=== PHASE 1: DATA INGESTION & ADVANCED FLATTENING ===
Unpacking nested spending behaviors...
Data successfully loaded and flattened.
Dataset Shape: 502 records and 35 attributes.


,_id,processing_timestamp,applicant_info.full_name,applicant_info.email,applicant_info.ssn,applicant_info.ip_address,applicant_info.gender,applicant_info.date_of_birth,applicant_info.zip_code,financials.annual_income,...,spending_Fitness,spending_Gambling,spending_Groceries,spending_Healthcare,spending_Insurance,spending_Rent,spending_Shopping,spending_Transportation,spending_Travel,spending_Utilities
0,app_200,2024-01-15T00:00:00Z,Jerry Smith,jerry.smith17@hotmail.com,596-64-4340,192.168.48.155,Male,2001-03-09,10036,73000,...,0,0,0,0,0,790,480,0,0,0
1,app_037,NaN,Brandon Walker,brandon.walker2@yahoo.com,425-69-4784,10.1.102.112,M,1992-03-31,10032,78000,...,0,0,0,243,0,608,0,0,0,0
2,app_215,NaN,Scott Moore,scott.moore94@mail.com,370-78-5178,10.240.193.250,Male,1989-10-24,10075,61000,...,0,0,0,0,0,109,0,0,0,0
3,app_024,NaN,Thomas Lee,thomas.lee6@protonmail.com,194-35-1833,192.168.175.67,Male,1983-04-25,10077,103000,...,575,0,0,0,0,0,0,0,0,0
4,app_184,2024-01-15T00:00:00Z,Brian Rodriguez,brian.rodriguez86@aol.com,480-41-2475,172.29.125.105,M,1999-05-21,10080,57000,...,0,0,0,0,0,0,0,0,0,0


## Phase 2: Systematic Data Profiling (Discovery)
In the real world, we cannot assume we know the data's flaws. We must build systematic checks to discover issues across the four dimensions of Data Quality.

1. **Completeness Profiler**: Standard `.isnull()` checks, plus a scanner for "hidden" nulls (empty strings, whitespace, 'N/A'). Furthermore we run a check to see if there are any useless columns (columns that have exactly 1 unique value)
2. **Consistency Profiler**: Checking data types against expected schemas and analyzing unique values in categorical fields to spot variations (e.g., 'M' vs 'Male').
3. **Validity & Accuracy Profiler**: Using statistical summaries to find impossible values (e.g., negative ages) and checking for logical duplicates (e.g., same SSN for different users).

In [4]:
print("=== 1. COMPLETENESS DISCOVERY ===")
# Standard nulls
standard_nulls = df.isnull().sum()

# Scanner for hidden nulls (strings that are just spaces or empty)
hidden_nulls = df.map(lambda x: str(x).strip() == '').sum()

completeness_df = pd.DataFrame({
    'Standard Nulls': standard_nulls,
    'Hidden Nulls (Empty Strings)': hidden_nulls,
    'Total Missing': standard_nulls + hidden_nulls
})
print("Columns with missing data detected:")
display(completeness_df[completeness_df['Total Missing'] > 0].sort_values(by='Total Missing', ascending=False))

# Check if any column has exactly 1 unique value (e.g., all 0s)
zero_variance_cols = [col for col in df.columns if df[col].nunique() <= 1]

if zero_variance_cols:
    print(f"Found {len(zero_variance_cols)} columns with zero variance (no useful information):")
    print(zero_variance_cols)
else:
    print("No zero-variance columns found.")


print("\n=== 2. CONSISTENCY DISCOVERY ===")
# Check data types to find mismatches (e.g., numbers stored as strings)
print("Data Types Overview:")
print(df.dtypes[df.dtypes == 'object']) # Focusing on object/string columns

# Check unique values for categorical columns to spot formatting inconsistencies
categorical_cols = ['applicant_info.gender', 'decision.rejection_reason']
for col in categorical_cols:
    if col in df.columns:
        print(f"\nUnique values in '{col}':")
        print(df[col].dropna().unique())


print("\n=== 3. VALIDITY & ACCURACY DISCOVERY ===")
# Statistical summary to spot impossible min/max values (e.g., negatives)
numeric_cols = df.select_dtypes(include=[np.number]).columns
print("Statistical Summary for Numeric Columns (Look at min/max):")
display(df[numeric_cols].describe().T[['min', 'max', 'mean']])

# Logical Duplicates Check (SSN is a primary identifier, it should be unique)
if 'applicant_info.ssn' in df.columns:
    duplicate_ssns = df[df.duplicated(subset=['applicant_info.ssn'], keep=False)]
    print(f"\nFound {duplicate_ssns['applicant_info.ssn'].nunique()} unique SSNs that are shared across {len(duplicate_ssns)} different records!")

=== 1. COMPLETENESS DISCOVERY ===
Columns with missing data detected:


,Standard Nulls,Hidden Nulls (Empty Strings),Total Missing
notes,500,0,500
financials.annual_salary,497,0,497
loan_purpose,452,0,452
processing_timestamp,440,0,440
decision.rejection_reason,292,0,292
decision.approved_amount,210,0,210
decision.interest_rate,210,0,210
applicant_info.email,0,7,7
applicant_info.ip_address,5,0,5
applicant_info.ssn,5,0,5


No zero-variance columns found.

=== 2. CONSISTENCY DISCOVERY ===
Data Types Overview:
financials.annual_income    object
dtype: object

Unique values in 'applicant_info.gender':
<StringArray>
['Male', 'M', 'F', 'Female', '']
Length: 5, dtype: str

Unique values in 'decision.rejection_reason':
<StringArray>
[       'algorithm_risk_score', 'insufficient_credit_history',
              'high_dti_ratio',                  'low_income']
Length: 4, dtype: str

=== 3. VALIDITY & ACCURACY DISCOVERY ===
Statistical Summary for Numeric Columns (Look at min/max):


,min,max,mean
financials.credit_history_months,-10.00,133.00,50.402390
financials.debt_to_income,0.05,1.85,0.246195
financials.savings_balance,-5000.00,88078.00,29493.503984
decision.interest_rate,2.50,6.50,4.564726
decision.approved_amount,15000.00,80000.00,47845.890411
financials.annual_salary,45000.00,94000.00,69200.000000
spending_Adult Entertainment,0.00,848.00,5.888446
spending_Alcohol,0.00,757.00,10.454183
spending_Dining,0.00,936.00,64.023904
spending_Education,0.00,889.00,63.701195



Found 3 unique SSNs that are shared across 11 different records!


### Phase 2.5: Deep Dive Profiling (Domain-Specific Checks)
To ensure maximum Data Quality, we must go beyond basic statistical profiling and check domain-specific business rules:

4. **Format Validation**: Using Regex to ensure SSNs, Emails, and IPs follow standard patterns.
5. **Cross-Field Logic**: Ensuring the loan decision logic (`approved` vs `rejected` fields) does not contradict itself.
6. **Nested Array Inspection**: Unpacking the `spending_behavior` array to check for anomalous spending amounts.
7. **Temporal Logic**: Checking for impossible birth dates.
8. **Final verification**: Checking for exact duplicate records, inconsistent data types and date formats.

In [5]:
print("=== 4. FORMAT VALIDITY (REGEX CHECKS) ===")
# Check for malformed emails
email_regex = r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$"
if 'applicant_info.email' in df.columns:
    valid_emails = df['applicant_info.email'].dropna().apply(lambda x: bool(re.match(email_regex, str(x))))
    invalid_emails = df['applicant_info.email'].dropna()[~valid_emails]
    print(f"Found {len(invalid_emails)} malformed emails.")
    if len(invalid_emails) > 0: print(invalid_emails.head())

# Check for malformed SSNs (expecting XXX-XX-XXXX)
ssn_regex = r"^\d{3}-\d{2}-\d{4}$"
if 'applicant_info.ssn' in df.columns:
    valid_ssns = df['applicant_info.ssn'].dropna().apply(lambda x: bool(re.match(ssn_regex, str(x))))
    invalid_ssns = df['applicant_info.ssn'].dropna()[~valid_ssns]
    print(f"\nFound {len(invalid_ssns)} malformed SSNs.")


print("\n=== 5. CROSS-COLUMN LOGICAL VALIDITY ===")
# Contradiction: Loan approved BUT has a rejection reason
contradiction_1 = df[(df['decision.loan_approved'] == True) & (df['decision.rejection_reason'].notnull())]
print(f"Contradiction (Approved but rejected): {len(contradiction_1)} rows")

# Contradiction: Loan rejected BUT has an approved amount
contradiction_2 = df[(df['decision.loan_approved'] == False) & (df['decision.approved_amount'].notnull())]
print(f"Contradiction (Rejected but has approved amount): {len(contradiction_2)} rows")


print("\n=== 6. PIVOTED DATA INSPECTION (SPENDING BEHAVIOR) ===")
# Since we exploded and pivoted the spending in Phase 1, 
# we now check all the new numerical spending columns for impossible negative values.
spending_cols = [col for col in df.columns if str(col).startswith('spending_')]
total_negative_spends = 0

for col in spending_cols:
    # Count how many negative values exist in this specific spending category
    negative_count = (df[col] < 0).sum()
    if negative_count > 0:
        print(f"  -> Found {negative_count} negative amounts in '{col}'")
        total_negative_spends += negative_count

print(f"Total negative spending amounts found across all categories: {total_negative_spends}")


print("\n=== 7. TEMPORAL ACCURACY ===")
if 'applicant_info.date_of_birth' in df.columns:
    # Convert to datetime just for checking (handling mixed formats safely with coerce to catch unparseable ones)
    temp_dob = pd.to_datetime(df['applicant_info.date_of_birth'], format='mixed', errors='coerce')
    
    # Check for dates in the future
    future_dates = temp_dob[temp_dob > pd.Timestamp.now()]
    print(f"Found {len(future_dates)} birth dates in the future.")
    
    # Check for unrealistic ages (e.g., born after 2008 -> under 18)
    underage = temp_dob[temp_dob > pd.Timestamp('2008-01-01')]
    print(f"Found {len(underage)} applicants appearing to be underage (< 18).")


print("\n=== 8. FINANCIAL VALIDITY (NEGATIVE BALANCES/RATIOS) ===")
# List of financial columns that strictly cannot be negative
financial_cols = [
    'financials.savings_balance', 
    'financials.debt_to_income', 
    'financials.credit_history_months'
]

for col in financial_cols:
    if col in df.columns:
        # We use pd.to_numeric with coerce just in case there are still strings
        temp_col = pd.to_numeric(df[col], errors='coerce')
        negative_count = (temp_col < 0).sum()
        if negative_count > 0:
            print(f"  -> ALARM: Found {negative_count} negative values in '{col}'!")


print("\n=== 9. FINAL VERIFICATION ===")

# 1. Exact Duplicate Records
# Checking if any entire row is a 1:1 copy of another
exact_duplicates = df[df.duplicated(subset=["_id"], keep=False)]
print(f"Found {len(exact_duplicates)} exact duplicate records.")

# 2. Inconsistent Data Types Across Records
# Checking the Python type of each individual cell in a problematic column
if 'financials.annual_income' in df.columns:
    print("\nRow-by-row data types in 'annual_income':")
    type_counts = df['financials.annual_income'].dropna().apply(type).value_counts()
    print(type_counts)

# 3. Inconsistent Date Formats
# Printing a sample to visually prove the formatting clash
if 'applicant_info.date_of_birth' in df.columns:
    print("\nSample of Inconsistent Date Formats in 'date_of_birth':")
    # Grabbing a mix of rows to show the discrepancy
    sample_dates = df['applicant_info.date_of_birth'].dropna().iloc[[0, 1, 5, 6]].values
    print(sample_dates)

=== 4. FORMAT VALIDITY (REGEX CHECKS) ===
Found 11 malformed emails.
26                           
138    mike johnson@gmail.com
181     test.user.outlook.com
187                          
275                          
Name: applicant_info.email, dtype: str

Found 0 malformed SSNs.

=== 5. CROSS-COLUMN LOGICAL VALIDITY ===
Contradiction (Approved but rejected): 0 rows
Contradiction (Rejected but has approved amount): 0 rows

=== 6. PIVOTED DATA INSPECTION (SPENDING BEHAVIOR) ===
Total negative spending amounts found across all categories: 0

=== 7. TEMPORAL ACCURACY ===
Found 0 birth dates in the future.
Found 0 applicants appearing to be underage (< 18).

=== 8. FINANCIAL VALIDITY (NEGATIVE BALANCES/RATIOS) ===
  -> ALARM: Found 1 negative values in 'financials.savings_balance'!
  -> ALARM: Found 2 negative values in 'financials.credit_history_months'!

=== 9. FINAL VERIFICATION ===
Found 4 exact duplicate records.

Row-by-row data types in 'annual_income':
financials.annual_income
<c

## Phase 3: Data Remediation (Cleaning Pipeline)
Based on our systematic profiling, we will now apply a data cleaning pipeline to resolve all identified issues, ensuring the dataset is ready for the Bias Detection and Privacy phases.

**Remediation Strategies by Dimension:**
1. **Accuracy (Duplicates)**: Drop exact row duplicates. Because of unhashable lists, we will use string conversion for the duplication mask.
2. **Consistency (Formatting & Schema)**: 
   - Standardize `gender` to 'Male' and 'Female'.
   - Consolidate the misaligned `annual_salary` column into `annual_income`.
   - Cast `annual_income` to numeric float values.
   - Standardize `date_of_birth` to uniform pandas `datetime` objects.
3. **Validity (Impossible Values)**:
   - Convert negative values in core financial columns to `NaN` (treating them as data entry errors).
   - Clean the nested `spending_behavior` arrays by removing dictionaries with negative amounts.
4. **Completeness (Imputation)**:
   - Fill missing numeric values (like the newly created NaNs in credit history or missing income) with the median to avoid losing records.

In [ ]:
print("Starting Data Remediation Pipeline...")

# Make a copy to preserve the original flattened dataframe
df_clean = df.copy()

# 1. ACCURACY: Drop Exact Duplicates
# We use the string casting trick to find and drop exact duplicate rows
duplicate_mask = df_clean.duplicated(subset=["_id"], keep='first')
df_clean = df_clean[~duplicate_mask]
print(f"Dropped {duplicate_mask.sum()} exact duplicate rows.")


# 2. CONSISTENCY: Schema, Types, and Formatting
# Schema consolidation
if 'financials.annual_salary' in df_clean.columns:
    df_clean['financials.annual_income'] = df_clean['financials.annual_income'].fillna(df_clean['financials.annual_salary'])
    df_clean.drop(columns=['financials.annual_salary'], inplace=True)
    print("Merged 'annual_salary' into 'annual_income' and dropped redundant column.")

# Type casting
df_clean['financials.annual_income'] = pd.to_numeric(df_clean['financials.annual_income'], errors='coerce')
print("Cast 'annual_income' to numeric.")

# Gender standardization
df_clean['applicant_info.gender'] = df_clean['applicant_info.gender'].replace({'M': 'Male', 'F': 'Female'})
print("Standardized 'gender' formats.")

# Date standardization
df_clean['applicant_info.date_of_birth'] = pd.to_datetime(df_clean['applicant_info.date_of_birth'], format='mixed', errors='coerce')
print("Standardized 'date_of_birth' to datetime objects.")


# 3. VALIDITY: Impossible Values
print("Fixing impossible validity errors...")

# Fix negative values in core financial columns by setting them to NaN (to be imputed later)
financial_cols_to_check = [
    'financials.savings_balance', 
    'financials.debt_to_income', 
    'financials.credit_history_months'
]

for col in financial_cols_to_check:
    if col in df_clean.columns:
        # Identify negative values
        invalid_mask = df_clean[col] < 0
        if invalid_mask.sum() > 0:
            df_clean.loc[invalid_mask, col] = np.nan
            print(f"Replaced {invalid_mask.sum()} negative values in '{col}' with NaN.")

# 4. COMPLETENESS: Handling Missing Data
# We must update our imputation list to include ALL financial columns so those new NaNs get fixed!
numeric_cols_to_impute = [
    'financials.annual_income', 
    'financials.credit_history_months',
    'financials.savings_balance',
    'financials.debt_to_income'
]

for col in numeric_cols_to_impute:
    if col in df_clean.columns:
        median_val = df_clean[col].median()
        df_clean[col] = df_clean[col].fillna(median_val)
print("Imputed missing numeric values (including fixed negative balances) with column medians.")

# Categorical missing values (e.g., gender, missing emails) 
# We'll label missing categorical values as 'Unknown' to explicitly track them
df_clean['applicant_info.gender'] = df_clean['applicant_info.gender'].fillna('Unknown')

print("\n=== REMEDIATION COMPLETE ===")
print(f"Final Cleaned Dataset Shape: {df_clean.shape[0]} records and {df_clean.shape[1]} attributes.")
display(df_clean.info())
display(df_clean.head(5))


Starting Data Remediation Pipeline...
Dropped 2 exact duplicate rows.
Merged 'annual_salary' into 'annual_income' and dropped redundant column.
Cast 'annual_income' to numeric.
Standardized 'gender' formats.
Standardized 'date_of_birth' to datetime objects.
Fixing impossible validity errors...
✔ Replaced 1 negative values in 'financials.savings_balance' with NaN.
✔ Replaced 2 negative values in 'financials.credit_history_months' with NaN.
Imputed missing numeric values (including fixed negative balances) with column medians.

=== REMEDIATION COMPLETE ===
Final Cleaned Dataset Shape: 500 records and 34 attributes.
<class 'pandas.DataFrame'>
Index: 500 entries, 0 to 501
Data columns (total 34 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   _id                               500 non-null    str           
 1   processing_timestamp              62 non-null     str           
 2   appli

None

,_id,processing_timestamp,applicant_info.full_name,applicant_info.email,applicant_info.ssn,applicant_info.ip_address,applicant_info.gender,applicant_info.date_of_birth,applicant_info.zip_code,financials.annual_income,...,spending_Fitness,spending_Gambling,spending_Groceries,spending_Healthcare,spending_Insurance,spending_Rent,spending_Shopping,spending_Transportation,spending_Travel,spending_Utilities
0,app_200,2024-01-15T00:00:00Z,Jerry Smith,jerry.smith17@hotmail.com,596-64-4340,192.168.48.155,Male,2001-03-09,10036,73000.0,...,0,0,0,0,0,790,480,0,0,0
1,app_037,NaN,Brandon Walker,brandon.walker2@yahoo.com,425-69-4784,10.1.102.112,Male,1992-03-31,10032,78000.0,...,0,0,0,243,0,608,0,0,0,0
2,app_215,NaN,Scott Moore,scott.moore94@mail.com,370-78-5178,10.240.193.250,Male,1989-10-24,10075,61000.0,...,0,0,0,0,0,109,0,0,0,0
3,app_024,NaN,Thomas Lee,thomas.lee6@protonmail.com,194-35-1833,192.168.175.67,Male,1983-04-25,10077,103000.0,...,575,0,0,0,0,0,0,0,0,0
4,app_184,2024-01-15T00:00:00Z,Brian Rodriguez,brian.rodriguez86@aol.com,480-41-2475,172.29.125.105,Male,1999-05-21,10080,57000.0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# Save the cleaned dataset to the processed folder
output_path = '../data/processed/cleaned_credit_applications.json'

# Convert datetime back to string for JSON serialization
df_export = df_clean.copy()
df_export['applicant_info.date_of_birth'] = df_export['applicant_info.date_of_birth'].dt.strftime('%Y-%m-%d')

# Export to JSON
df_export.to_json(output_path, orient='records', indent=4)
print(f"Cleaned dataset successfully exported to: {output_path}")

Cleaned dataset successfully exported to: ../data/processed/cleaned_credit_applications.json
